In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import json

/var/folders/hb/9qr23dms0lg2_mgzzyhsqyhh0000gn/T/ipykernel_13019/348472887.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
#read in companies to use 
companies_df = pd.read_csv("../datasets/constituents.csv")
company_tickers = companies_df["Symbol"].tolist()

In [5]:
# Initialise two dataframes
company_df = pd.DataFrame(columns=['id', 'name', 'fullTimeEmployees', 'Industry', 'Country'])
finance_situation_df = pd.DataFrame(columns=['company_id', 'AuditRisk', 'Dividend rate', 'Dividend Yield', 
                                             'Payout rate', 'Beta', 'Market Cap', 'profit margins', 'short ratio', 
                                             'quick ratio', 'current ratio', 'debtToEquity'])


In [6]:
# Retrieving information and append to data frame for each ticker
for ticker in company_tickers:
    try:
        company_dict = {"id":ticker}
        finance_dict = {"company_id":ticker}
        company = yf.Ticker(ticker)
        information = company.info
        print(information)
        company_dict.update({'name':information.get('shortName', None), 'fullTimeEmployees':information.get('fullTimeEmployees', None),
                             'Industry':information.get('sector', None), 'Country':information.get('country', None)})
        company_df = pd.concat([company_df, pd.DataFrame([company_dict])], ignore_index=True)
        finance_dict.update({'AuditRisk': information.get('auditRisk', None), 
                             'Dividend rate': information.get('dividendRate', None), 
                             'Dividend Yield': information.get('dividendYield', None),
                             'Payout rate': information.get('payoutRatio', None), 
                             'Beta': information.get('beta', None), 
                             'Market Cap': information.get('marketCap', None),
                             'profit margins': information.get('profitMargins', None), 
                             'short ratio': information.get('shortRatio', None),
                             'quick ratio': information.get('quickRatio', None), 
                             'current ratio': information.get('currentRatio', None),
                             'debtToEquity': information.get('debtToEquity', None)})
        finance_situation_df = pd.concat([finance_situation_df, pd.DataFrame([finance_dict])], ignore_index=True)
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")

{'address1': '1600 Amphitheatre Parkway', 'city': 'Mountain View', 'state': 'CA', 'zip': '94043', 'country': 'United States', 'phone': '650 253 0000', 'website': 'https://abc.xyz', 'industry': 'Internet Content & Information', 'industryKey': 'internet-content-information', 'industryDisp': 'Internet Content & Information', 'sector': 'Communication Services', 'sectorKey': 'communication-services', 'sectorDisp': 'Communication Services', 'longBusinessSummary': 'Alphabet Inc. offers various products and platforms in the United States, Europe, the Middle East, Africa, the Asia-Pacific, Canada, and Latin America. It operates through Google Services, Google Cloud, and Other Bets segments. The Google Services segment provides products and services, including ads, Android, Chrome, devices, Gmail, Google Drive, Google Maps, Google Photos, Google Play, Search, and YouTube. It is also involved in the sale of apps and in-app purchases and digital content in the Google Play and YouTube; and devices,

/var/folders/hb/9qr23dms0lg2_mgzzyhsqyhh0000gn/T/ipykernel_13019/975602517.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finance_situation_df = pd.concat([finance_situation_df, pd.DataFrame([finance_dict])], ignore_index=True)


{'address1': '280 Park Avenue', 'city': 'New York', 'state': 'NY', 'zip': '10017', 'country': 'United States', 'phone': '212 415 3600', 'fax': '212 415 3530', 'website': 'https://www.omnicomgroup.com', 'industry': 'Advertising Agencies', 'industryKey': 'advertising-agencies', 'industryDisp': 'Advertising Agencies', 'sector': 'Communication Services', 'sectorKey': 'communication-services', 'sectorDisp': 'Communication Services', 'longBusinessSummary': "Omnicom Group Inc., together with its subsidiaries, offers advertising, marketing, and corporate communications services. It provides a range of services in the areas of advertising and media, precision marketing, commerce and branding, experiential, execution and support, public relations, and healthcare. The company's services include advertising, branding, content marketing, corporate social responsibility consulting, crisis communications, custom publishing, data analytics, database management, digital/direct marketing and post-produc

KeyboardInterrupt: 

In [5]:
# Convert data types
company_df['fullTimeEmployees'] = company_df['fullTimeEmployees'].astype(float) 
columns = finance_situation_df.columns.tolist()
finance_situation_df[columns[1:]]= finance_situation_df[columns[1:]].astype(float)

In [6]:
# calculate market cap ratio
merged_df = pd.merge(company_df, finance_situation_df, left_on='id', right_on='company_id')
industry_market_cap = merged_df.groupby('Industry')['Market Cap'].sum()
finance_situation_df['Market Cap'] = merged_df.apply(lambda row: (row['Market Cap'] / industry_market_cap[row['Industry']]) * 100, axis=1)

In [7]:
# store 2 csv files
company_df.to_csv('Company.csv', index=False) 
finance_situation_df.to_csv('../Finance_Situation/Finance_Situation.csv', index = False)

In [9]:
companies_df = pd.read_csv("/Users/wanghaojin/Desktop/exchange@NUS/IS3107/HW/Project/news_data_pipeline/datasets/constituents.csv")
#read in companies to use 
company_tickers = companies_df["Symbol"].tolist()
for ticker in company_tickers:
    try:
        company = yf.Ticker(ticker)
        information = company.info
        with open('/Users/wanghaojin/Desktop/exchange@NUS/IS3107/HW/Project/news_data_pipeline/Company/raw_company.json','a') as f:
            json.dump(information,f,indent=4)
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")